Each jpeg has the raw image on the left and the semantic segmentation result on the right.  

![Example](https://i.imgur.com/50UFABF.jpg)

This example shows how to split the images into separate TensorFlow iterators.


### Imports

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
# import tensorflow.contrib.eager as tfe

# tf.enable_eager_execution()


### Functions to Read Image and Split Them

In [36]:
def _read_to_tensor(fname, output_height=256, output_width=512, normalize_data=False):
    '''Original images are 256 x 512 x 3. Left half is original image, right is semantic seg'''
    img_strings = tf.io.read_file(fname)
    imgs_decoded = tf.image.decode_jpeg(img_strings)
    output = tf.image.resize(imgs_decoded, [output_height, output_width])
    if normalize_data:
        output = (output - 128) / 128
    return output

def _get_left_img_half(inp, width=256):
    return inp[:, :width, :]

def _get_right_img_half(inp, width=256):
    return inp[:, width:, :]
    

### Get List of Files and Apply Functions Above to Create Dataset

In [37]:
drive_dir = '/content/drive/My Drive/CS 230/Project/Canny Detector/cityscapes'
img_dir = drive_dir + '/train'
file_list = os.listdir(img_dir)
img_paths = [os.path.join(img_dir, fname) for fname in file_list]

# Start with a dataset of directory names.
output_height = 256
output_width = 256
my_data = tf.data.Dataset.from_tensor_slices(img_paths)
img_tensors = my_data.map(_read_to_tensor)
left_imgs = img_tensors.map(_get_left_img_half)
right_imgs = img_tensors.map(_get_right_img_half)

### Simple Demo That The Images Have Been Read

In [42]:
import imageio

left_batches = tf.compat.v1.data.make_one_shot_iterator(left_imgs)  # outside of TF Eager, we would use make_one_shot_iterator
right_batches = tf.compat.v1.data.make_one_shot_iterator(right_imgs)
output_path = drive_dir + '/output/'
n_images_to_show = 2975

for i in range(n_images_to_show):
    left_img = left_batches.next().numpy().astype(np.uint8)
    right_img = right_batches.next().numpy().astype(np.uint8)
    # fig = plt.figure()
    # fig.add_subplot(1,2,1)
    # plt.imshow(left_img)
    # fig.add_subplot(1,2,2)
    # plt.imshow(right_img)
    # plt.show()
    imageio.imwrite(output_path + 'images/' + str(i) + '.jpg', left_img)
    imageio.imwrite(output_path + 'labels/' + str(i) + '.jpg', right_img)

    if i % 200 == 0:
      print(i)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
